In [ ]:
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications import VGG16;
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

IMG_WIDTH = 32
IMG_HEIGHT = 32
IMG_DEPTH = 1
BATCH_SIZE = 16

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
classes = np.unique(y_train)
num_classes = len(classes)
num_classes

In [ ]:
x_train = x_train.reshape(-1, 784)
x_test= x_test.reshape (-1, 784)
x_train.shape, x_test.shape

In [ ]:
x_train = np.dstack([x_train] * 3)
x_test = np.dstack([x_test] * 3)
x_train.shape, x_test.shape

In [ ]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_test= x_test.reshape (-1, 28, 28, 1)
x_train.shape, x_test.shape

In [ ]:
# Resize the images to 48 * 48 as required by VGG16
x_train = np.asarray([img_to_array(array_to_img(im, scale=False).resize((32, 32))) for im in x_train])
x_test = np.asarray([img_to_array(array_to_img(im, scale=False).resize((32, 32))) for im in x_test])
x_train.shape, x_test.shape

In [ ]:
# Normalise the data and change data type
x_train = x_train / 255.
x_test = x_test / 255.
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [ ]:
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [ ]:
x_train, x_valid, train_label, valid_label = train_test_split(x_train, y_train_one_hot, test_size=0.2, random_state=13)
x_train.shape, x_valid.shape, train_label.shape, valid_label.shape

In [ ]:
# Preprocessing the input 
x_train = preprocess_input(x_train)
x_valid = preprocess_input(x_valid)
x_test  = preprocess_input (x_test)

In [ ]:
#  Create base model of VGG16
conv_base = VGG16(weights='vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH))
conv_base.summary()

In [ ]:
# Extracting features
train_features = conv_base.predict(np.array(x_train), batch_size=BATCH_SIZE, verbose=1)
test_features = conv_base.predict(np.array(x_test), batch_size=BATCH_SIZE, verbose=1)
val_features = conv_base.predict(np.array(x_valid), batch_size=BATCH_SIZE, verbose=1)

In [ ]:
#Saving the features so that they can be used for future
np.savez("train_features", train_features, train_label)
np.savez("test_features", test_features, y_test)
np.savez("val_features", val_features, valid_label)

In [ ]:
#Current shape of features
print(train_features.shape, "\n",  test_features.shape, "\n", val_features.shape)

In [ ]:
# Flatten extracted features
train_features_flat = np.reshape(train_features, (48000, 1 * 1 * 512))
test_features_flat = np.reshape(test_features, (10000, 1 * 1 * 512))
val_features_flat = np.reshape(val_features, (12000, 1 * 1 * 512))

In [ ]:
NB_TRAIN_SAMPLES = train_features_flat.shape[0]
NB_VALIDATION_SAMPLES = val_features_flat.shape[0]
NB_EPOCHS = 100

model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_dim=(1*1*512)))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(), metrics=['acc'])

In [ ]:
reduce_learning = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2,
    verbose=1,
    mode='auto',
    epsilon=0.0001,
    cooldown=2,
    min_lr=0)

eary_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=7,
    verbose=1,
    mode='auto')

callbacks = [reduce_learning, eary_stopping]

In [ ]:
history = model.fit(
    train_features_flat,
    train_label,
    epochs=NB_EPOCHS,
    validation_data=(val_features_flat, valid_label),
    callbacks=callbacks
)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.title('Training and validation accuracy')
plt.plot(epochs, acc, 'red', label='Training acc')
plt.plot(epochs, val_acc, 'blue', label='Validation acc')
plt.legend()

plt.figure()
plt.title('Training and validation loss')
plt.plot(epochs, loss, 'red', label='Training loss')
plt.plot(epochs, val_loss, 'blue', label='Validation loss')

plt.legend()

plt.show()